In [43]:
from langchain_openai import AzureChatOpenAI
import os
import json
def return_metadata(path):
    try:
        # Set environment variables
        os.environ["OPENAI_API_VERSION"] = ""  # Ensure this is correct
        os.environ["AZURE_OPENAI_ENDPOINT"] = ""  # Base URL only
        os.environ["AZURE_OPENAI_API_KEY"] = ""  # Correct API key

        # Define the deployment ID (the actual model deployment in Azure)
        deployment_id = "gpt4"  # Ensure this matches the model deployed in Azure

        # Create an instance of Azure Chat OpenAI
        llm = AzureChatOpenAI(
            deployment_name=deployment_id,  # Use the actual deployment name here
            openai_api_version=os.environ["OPENAI_API_VERSION"],
            api_key=os.environ["AZURE_OPENAI_API_KEY"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
        )

        prompt_template=f'''generate meta-title and detailed meta-description as string for {path} include high search volume keywords and make seo optimized . Meta description should be a paragraph can be long with limit 150- 160 but limit meta title to 60 letters'''+ '''Return the output format should be like 
        {
                    "meta_title": "",
                    "meta_description": "",
                }. Be concise in your response. Don't return any text only return json'''
        # Example prompt
        messages = [{"role": "user", "content": prompt_template}]  # Format for chat models

        # Invoke the model
        result = llm.invoke(messages)
        json_string=result.content.replace('json','').replace('`','')
        json_object = json.loads(json_string)
        return json_object
    except Exception as e:
        print(e)
        return {
                    "meta_title": '',
                    "meta_description": ''
                }

In [44]:
import json

# Path to your JSON file
file_path = 'categories_reference.json'  # Adjust the path if necessary

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)   

paths = []

def find_paths(d, current_path=None):
    if current_path is None:
        current_path = []
    
    for k, v in d.items():
        new_path = current_path + [k]  # Create a new path as a list
        if isinstance(v, dict) and v:  # Check if the value is a non-empty dict
            find_paths(v, new_path)
        else:  # Leaf node (empty dict)
            paths.append(new_path)   

In [45]:
find_paths(data)
path_str=[' > '.join(path) for path in paths]

In [46]:
a=[]
for path in path_str:
    print(f"Processing:{path}")
    metadata= return_metadata(path)
    a.append({"name":path, "metadata":metadata})

Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge in same P - SKU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge in same P - SPU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge cross P - SKU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge cross P - SPU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P and PV merge - SKU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P and PV merge - SPU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P merge - SKU
Processing:Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P merge - SPU
Processing:Test Product （wont be seen in seller pls dont inactive) > wxj test none lea

In [47]:
import json
with open('metadata.json', 'w') as json_file:
    json.dump(a, json_file)